In [ ]:
#DES ENCRYPTION AND DECRYPTION 
def string_to_padded_hex(s):
    hex_string = s.encode('utf-8').hex()  # Convert string to hex
    hex_length = len(hex_string)
    
    # Each character in hex represents 4 bits, so 64 bits = 16 hex characters
    remainder = hex_length % 16
    if remainder != 0:
        padding_needed = 16 - remainder
        hex_string = hex_string.ljust(hex_length + padding_needed, '0')  # Pad with 0s
    
    return hex_string

text = "0123456789ABCDEF"
hex_output = string_to_padded_hex(text)
print(hex_output)

30313233343536373839414243444546


In [ ]:
def Message_split(hex_string):
    if len(hex_string) != 16:
        raise ValueError("Input hex string must be exactly 16 characters (64 bits).")
    
    # Convert hex to binary and remove the '0b' prefix
    binary_string = bin(int(hex_string, 16))[2:]
    
    # Ensure it's exactly 64 bits (pad with leading zeros if necessary)
    a=binary_string.zfill(64)
    return a[0:32],a[32:]
hex_input = text  # Hex representation of 'Hello' with padding
a,b = Message_split(hex_input)
print(a,b)
print(len(a),len(b))
print(a+b)

00000001001000110100010101100111 10001001101010111100110111101111
32 32
0000000100100011010001010110011110001001101010111100110111101111


In [240]:
PC1=[         57 ,  49  ,  41 ,  33   , 25  ,  17  ,  9,
               1 ,  58   , 50 ,  42   , 34  ,  26  , 18,
              10 ,   2   , 59 ,  51   , 43  ,  35  , 27,
              19  , 11   ,  3  , 60   , 52  ,  44  , 36,
              63  , 55   , 47  , 39   , 31  ,  23  , 15,
               7  , 62   , 54  , 46   , 38  ,  30  , 22,
              14  ,  6   , 61  , 53  ,  45  ,  37  , 29,
              21  , 13    , 5  , 28  ,  20  ,  12  ,  4,
]
# print(len(PC1))
def Key(hex_string):
    if len(hex_string) != 16:
        raise ValueError("Input hex string must be exactly 16 characters (64 bits).")
    # Convert hex to binary and remove the '0b' prefix
    binary_string = bin(int(hex_string, 16))[2:].zfill(64)
    permuted_56 = ''.join(binary_string[i - 1] for i in PC1)
    return permuted_56[0:28],permuted_56[28:]
C0,D0=Key("133457799BBCDFF1")    
print(C0,D0)

1111000011001100101010101111 0101010101100110011110001111


In [47]:
left_shift_arr = [
    1, 1, 2, 2, 2, 2, 2, 2,
    1, 2, 2, 2, 2, 2, 2, 1
]

In [48]:
def left_shift(binary_str, shift_amount):
    shift_amount %= len(binary_str)  # Ensure shift doesn't exceed length
    return binary_str[shift_amount:] + binary_str[:shift_amount]

In [84]:
def generate_C1_to_C16(C0, D0):
    C = []  # Initialize an empty list
    # C.append(C0+D0)
    for i in range(16):
        if i == 0:
            new_C = left_shift(C0, left_shift_arr[i]) + left_shift(D0, left_shift_arr[i])
        else:
            new_C = left_shift(C[i - 1][:28], left_shift_arr[i]) + left_shift(C[i - 1][28:], left_shift_arr[i])
        C.append(new_C) 
    return C
C=generate_C1_to_C16(C0,D0)
print(C)

['11100001100110010101010111111010101011001100111100011110', '11000011001100101010101111110101010110011001111000111101', '00001100110010101010111111110101011001100111100011110101', '00110011001010101011111111000101100110011110001111010101', '11001100101010101111111100000110011001111000111101010101', '00110010101010111111110000111001100111100011110101010101', '11001010101011111111000011000110011110001111010101010110', '00101010101111111100001100111001111000111101010101011001', '01010101011111111000011001100011110001111010101010110011', '01010101111111100001100110011111000111101010101011001100', '01010111111110000110011001011100011110101010101100110011', '01011111111000011001100101010001111010101010110011001111', '01111111100001100110010101010111101010101011001100111100', '11111110000110011001010101011110101010101100110011110001', '11111000011001100101010101111010101010110011001111000111', '11110000110011001010101011110101010101100110011110001111']


In [85]:
PC2 = [
    14, 17, 11, 24,  1,  5,
     3, 28, 15,  6, 21, 10,
    23, 19, 12,  4, 26,  8,
    16,  7, 27, 20, 13,  2,
    41, 52, 31, 37, 47, 55,
    30, 40, 51, 45, 33, 48,
    44, 49, 39, 56, 34, 53,
    46, 42, 50, 36, 29, 32
]
# print(len(PC2))

In [92]:
def apply_permutation(binary_string, permutation_table):
    return ''.join(binary_string[i - 1] for i in permutation_table)

In [105]:
def generate_K1_to_K16(C):
    K=[]
    key_i=""
    for i in range(16):
        key_i+= apply_permutation(C[i],PC2)
        K.append(key_i)
        key_i=""
    return K   
K=generate_K1_to_K16(C)

In [107]:
print(K)
# print(len(K))

['000110110000001011101111111111000111000001110010', '011110011010111011011001110110111100100111100101', '010101011111110010001010010000101100111110011001', '011100101010110111010110110110110011010100011101', '011111001110110000000111111010110101001110101000', '011000111010010100111110010100000111101100101111', '111011001000010010110111111101100001100010111100', '111101111000101000111010110000010011101111111011', '111000001101101111101011111011011110011110000001', '101100011111001101000111101110100100011001001111', '001000010101111111010011110111101101001110000110', '011101010111000111110101100101000110011111101001', '100101111100010111010001111110101011101001000001', '010111110100001110110111111100101110011100111010', '101111111001000110001101001111010011111100001010', '110010110011110110001011000011100001011111110101']


In [113]:
IP_table = [
    58, 50, 42, 34, 26, 18, 10,  2,
    60, 52, 44, 36, 28, 20, 12,  4,
    62, 54, 46, 38, 30, 22, 14,  6,
    64, 56, 48, 40, 32, 24, 16,  8,
    57, 49, 41, 33, 25, 17,  9,  1,
    59, 51, 43, 35, 27, 19, 11,  3,
    61, 53, 45, 37, 29, 21, 13,  5,
    63, 55, 47, 39, 31, 23, 15,  7
]


In [114]:
IP_string=apply_permutation(a+b,IP_table)
print(IP_string)
# print(len(IP_string))

1100110000000000110011001111111111110000101010101111000010101010
64


In [118]:
L0=IP_string[0:32]
R0=IP_string[32:]

In [197]:
E_bit = [
    32,  1,  2,  3,  4,  5,
     4,  5,  6,  7,  8,  9,
     8,  9, 10, 11, 12, 13,
    12, 13, 14, 15, 16, 17,
    16, 17, 18, 19, 20, 21,
    20, 21, 22, 23, 24, 25,
    24, 25, 26, 27, 28, 29,
    28, 29, 30, 31, 32,  1
]
S_boxes = [
    # S1
    [
        [14,  4, 13,  1,  2, 15, 11,  8,  3, 10,  6, 12,  5,  9,  0,  7],
        [ 0, 15,  7,  4, 14,  2, 13,  1, 10,  6, 12, 11,  9,  5,  3,  8],
        [ 4,  1, 14,  8, 13,  6,  2, 11, 15, 12,  9,  7,  3, 10,  5,  0],
        [15, 12,  8,  2,  4,  9,  1,  7,  5, 11,  3, 14, 10,  0,  6, 13]
    ],
    # S2
    [
        [15,  1,  8, 14,  6, 11,  3,  4,  9,  7,  2, 13, 12,  0,  5, 10],
        [ 3, 13,  4,  7, 15,  2,  8, 14, 12,  0,  1, 10,  6,  9, 11,  5],
        [ 0, 14,  7, 11, 10,  4, 13,  1,  5,  8, 12,  6,  9,  3,  2, 15],
        [13,  8, 10,  1,  3, 15,  4,  2, 11,  6,  7, 12,  0,  5, 14,  9]
    ],
    # S3
    [
        [10,  0,  9, 14,  6,  3, 15,  5,  1, 13, 12,  7, 11,  4,  2,  8],
        [13,  7,  0,  9,  3,  4,  6, 10,  2,  8,  5, 14, 12, 11, 15,  1],
        [13,  6,  4,  9,  8, 15,  3,  0, 11,  1,  2, 12,  5, 10, 14,  7],
        [ 1, 10, 13,  0,  6,  9,  8,  7,  4, 15, 14,  3, 11,  5,  2, 12]
    ],
    # S4
    [
        [ 7, 13, 14,  3,  0,  6,  9, 10,  1,  2,  8,  5, 11, 12,  4, 15],
        [13,  8, 11,  5,  6, 15,  0,  3,  4,  7,  2, 12,  1, 10, 14,  9],
        [10,  6,  9,  0, 12, 11,  7, 13, 15,  1,  3, 14,  5,  2,  8,  4],
        [ 3, 15,  0,  6, 10,  1, 13,  8,  9,  4,  5, 11, 12,  7,  2, 14]
    ],
    # S5
    [
        [ 2, 12,  4,  1,  7, 10, 11,  6,  8,  5,  3, 15, 13,  0, 14,  9],
        [14, 11,  2, 12,  4,  7, 13,  1,  5,  0, 15, 10,  3,  9,  8,  6],
        [ 4,  2,  1, 11, 10, 13,  7,  8, 15,  9, 12,  5,  6,  3,  0, 14],
        [11,  8, 12,  7,  1, 14,  2, 13,  6, 15,  0,  9, 10,  4,  5,  3]
    ],
    # S6
    [
        [12,  1, 10, 15,  9,  2,  6,  8,  0, 13,  3,  4, 14,  7,  5, 11],
        [10, 15,  4,  2,  7, 12,  9,  5,  6,  1, 13, 14,  0, 11,  3,  8],
        [ 9, 14, 15,  5,  2,  8, 12,  3,  7,  0,  4, 10,  1, 13, 11,  6],
        [ 4,  3,  2, 12,  9,  5, 15, 10, 11, 14,  1,  7,  6,  0,  8, 13]
    ],
    # S7
    [
        [ 4, 11,  2, 14, 15,  0,  8, 13,  3, 12,  9,  7,  5, 10,  6,  1],
        [13,  0, 11,  7,  4,  9,  1, 10, 14,  3,  5, 12,  2, 15,  8,  6],
        [ 1,  4, 11, 13, 12,  3,  7, 14, 10, 15,  6,  8,  0,  5,  9,  2],
        [ 6, 11, 13,  8,  1,  4, 10,  7,  9,  5,  0, 15, 14,  2,  3, 12]
    ],
    # S8
    [
        [13,  2,  8,  4,  6, 15, 11,  1, 10,  9,  3, 14,  5,  0, 12,  7],
        [ 1, 15, 13,  8, 10,  3,  7,  4, 12,  5,  6, 11,  0, 14,  9,  2],
        [ 7, 11,  4,  1,  9, 12, 14,  2,  0,  6, 10, 13, 15,  3,  5,  8],
        [ 2,  1, 14,  7,  4, 10,  8, 13, 15, 12,  9,  0,  3,  5,  6, 11]
    ]
]
P_after_sbox = [
    16,  7, 20, 21,
    29, 12, 28, 17,
     1, 15, 23, 26,
     5, 18, 31, 10,
     2,  8, 24, 14,
    32, 27,  3,  9,
    19, 13, 30,  6,
    22, 11,  4, 25
]
IP_inv = [
    40,  8, 48, 16, 56, 24, 64, 32,
    39,  7, 47, 15, 55, 23, 63, 31,
    38,  6, 46, 14, 54, 22, 62, 30,
    37,  5, 45, 13, 53, 21, 61, 29,
    36,  4, 44, 12, 52, 20, 60, 28,
    35,  3, 43, 11, 51, 19, 59, 27,
    34,  2, 42, 10, 50, 18, 58, 26,
    33,  1, 41,  9, 49, 17, 57, 25
]

In [229]:
def S_func_i(B_i,S_i):
    row=int(B_i[0]+B_i[5],2)
    column=int(B_i[1:5],2)
    return bin(S_i[row][column])[2:].zfill(4)

In [231]:
def f_func(Rn_1, Kn,Ln_1):
    E_Rn_1 = apply_permutation(Rn_1, E_bit)
    xor_result = bin(int(E_Rn_1, 2) ^ int(Kn, 2))[2:].zfill(len(E_Rn_1))
    S_box_temp=""
    for i in range(0,len(xor_result),6):
      S_box_temp+=S_func_i(xor_result[i:i+6],S_boxes[i//6])
    f=apply_permutation(S_box_temp,P_after_sbox)
    return f;

In [233]:
def DES_encrypt(L0,R0):
  L,R=[],[]
  for i in range(16):
      # print(i)
      if(i==0):
        # pass  
        L.append(bin(int(R0,2))[2:].zfill(32))
        R.append(bin(int(L0,2)^int(f_func(R0,K[0],L0),2))[2:].zfill(32))  
      else:
        # pass  
        L.append(R[i-1])
        R.append(bin( int(L[i-1],2) ^ int(f_func(R[i-1],K[i],L[i-1]),2))[2:].zfill(32))  
  return   apply_permutation(R[15]+L[15],IP_inv)       
a=DES_encrypt(L0,R0)
print(a)


1000010111101000000100110101010000001111000010101011010000000101


In [235]:
def binary_to_hex(binary_string):
    # Ensure the binary string length is a multiple of 4 by padding with leading zeros
    binary_string = binary_string.zfill((len(binary_string) + 3) // 4 * 4)
    
    # Convert binary to hexadecimal
    hex_string = hex(int(binary_string, 2))[2:].upper()
    
    return hex_string

In [273]:
def generalized_DES_encrypt(input,key):
    a,b=Message_split(input)
    C0,D0=Key(key)
    C=generate_C1_to_C16(C0,D0)
    K=generate_K1_to_K16(C)
    IP_string=apply_permutation(a+b,IP_table)
    L0=IP_string[0:32]
    R0=IP_string[32:]
    a=DES_encrypt(L0,R0)
    return binary_to_hex(a)
cipher_text=generalized_DES_encrypt("123456789ABCDEE9","133457799BBCDFF1")    
print(cipher_text)

8EE3658640B9CE04


In [266]:
K1=K[::-1]

In [267]:
def DES_decrypt(L0,R0):
  L,R=[],[]
  for i in range(16):
      # print(i)
      if(i==0):
        # pass  
        L.append(bin(int(R0,2))[2:].zfill(32))
        R.append(bin(int(L0,2)^int(f_func(R0,K1[0],L0),2))[2:].zfill(32))  
      else:
        # pass  
        L.append(R[i-1])
        R.append(bin( int(L[i-1],2) ^ int(f_func(R[i-1],K1[i],L[i-1]),2))[2:].zfill(32))  
  return   apply_permutation(R[15]+L[15],IP_inv)      

In [274]:
def generalized_DES_decrypt(ciphertext, key):
    a, b = Message_split(ciphertext)  # Convert hex input to binary
    C0, D0 = Key(key)  
    C = generate_C1_to_C16(C0, D0)  
    K = generate_K1_to_K16(C)  # Generate 16 subkeys
    # print(len(K))
    # K1=K[::-1]
    IP_string = apply_permutation(a + b, IP_table)  
    L0 = IP_string[0:32]  
    R0 = IP_string[32:]  
    # Reverse key order for decryption
    decrypted_binary = DES_decrypt(L0, R0)  
    return binary_to_hex(decrypted_binary)  
plaintext = generalized_DES_decrypt(cipher_text, "133457799BBCDFF1")  
print(plaintext)  

123456789ABCDEE9
